In [1]:
import requests
import pandas as pd
import numpy as np

import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import Nominatim
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

import folium

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


CLIENT_ID = 'TS2DPJEPPDPN5EZMGNMTHTJ25T3P0NOZM0D4LRFEHUUX1J1D' # your Foursquare ID
CLIENT_SECRET = 'VASOSLOQETUPKYRXOCPNJWQSXPCXKK12XD3KUY2FK5P2RKHE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [58]:


url = 'https://www.geonames.org/postal-codes/AU/ACT/australian-capital-territory.html'
html = requests.get(url).content
df_table = pd.read_html(html)[-1]
# update column names
df_table.columns=['NaN', 'Neighbourhood','Code','Country','Admin1','Admin2','Admin3']
df_table = df_table.drop(columns=['NaN'])
df_table=df_table.iloc[1:]

df_table.head()

,Neighbourhood,Code,Country,Admin1,Admin2,Admin3
1,Braddon,2612,Australia,Australian Capital Territory,CANBERRA,NaN
2,-35.271/149.133,NaN,NaN,NaN,NaN,NaN
3,Kambah,2902,Australia,Australian Capital Territory,CANBERRA,NaN
4,-35.387/149.06,NaN,NaN,NaN,NaN,NaN
5,Deakin,2600,Australia,Australian Capital Territory,CANBERRA,NaN


In [66]:
locations = df_table.iloc[1::2, :]['Neighbourhood'].values
df_canberra = df_table.iloc[::2, :][['Neighbourhood','Code','Admin2']]
df_canberra['geo'] = locations
df_canberra[['Latitude','Longitude']] = df_canberra.geo.str.split("/",expand=True,)
df_canberra.drop(df_canberra[df_canberra['Admin2']!='CANBERRA'].index,inplace=True)
df_canberra=df_canberra.drop(columns=['geo','Admin2'])
df_canberra.set_index(['Neighbourhood'], inplace = True,append = True, drop = True) 
df_canberra.reset_index(inplace = True)
df_canberra=df_canberra.drop(columns=['level_0'])

df_canberra

,Neighbourhood,Code,Latitude,Longitude
0,Braddon,2612,-35.271,149.133
1,Kambah,2902,-35.387,149.06
2,Deakin,2600,-35.318,149.103
3,Yarralumla,2600,-35.307,149.102
4,Barton,2600,-35.309,149.136
5,City,2601,-35.281,149.129
6,Hackett,2602,-35.249,149.163
7,Dickson,2602,-35.254,149.138
8,O'Connor,2602,-35.258,149.12
9,Ainslie,2602,-35.262,149.147


In [68]:
address = 'Canberra'

geolocator = Nominatim(user_agent="canberra_explorer")
location = geolocator.geocode(address)
canberra_latitude = location.latitude
canberra_longitude = location.longitude
print('The geograpical coordinate of Canberra are {}, {}.'.format(canberra_latitude, canberra_longitude))

The geograpical coordinate of Canberra are -35.2975906, 149.1012676.


In [70]:
# create map of canberra using latitude and longitude values
map_canberra = folium.Map(location=[canberra_latitude, canberra_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_canberra['Latitude'], df_canberra['Longitude'], df_canberra['Code'], df_canberra['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canberra)

map_canberra